This notebook is designed to run in a IBM Watson Studio default runtime (NOT the Watson Studio Apache Spark Runtime as the default runtime with 1 vCPU). Therefore,  Apache Spark is installed in local mode (for test purposes only). We wont use it in production.

In case of facing issues, the following two documents could help me:

https://github.com/IBM/skillsnetwork/wiki/Environment-Setup

https://github.com/IBM/skillsnetwork/wiki/FAQ

Then, I can ask for help here:

https://coursera.org/learn/machine-learning-big-data-apache-spark/discussions/all

Before asking a question, I should  make sure to follow the guidelines:

https://github.com/IBM/skillsnetwork/wiki/FAQ#im-feeling-lost-and-confused-please-help-me


If running outside Watson Studio, this should work as well. In case I am running in an Apache Spark context outside Watson Studio, I must remove the Apache Spark setup in the first notebook cells.

In [ ]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown('# <span style="color:red">'+string+'</span>'))


if ('sc' in locals() or 'sc' in globals()):
    printmd('<<<<<!!!!! It seems that you are running in a IBM Watson Studio Apache Spark Notebook. Please run it in an IBM Watson Studio Default Runtime (without Apache Spark) !!!!!>>>>>')


In [ ]:
!pip install pyspark==2.4.5

In [ ]:
try:
    from pyspark import SparkContext, SparkConf
    from pyspark.sql import SparkSession
except ImportError as e:
    printmd('<<<<<!!!!! Please restart your kernel after installing Apache Spark !!!!!>>>>>')

In [ ]:
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

spark = SparkSession \
    .builder \
    .getOrCreate()

In case you want to learn how ETL is done, please run the following notebook first and update the file name below accordingly

https://github.com/IBM/coursera/blob/master/coursera_ml/a2_w1_s3_ETL.ipynb

In [ ]:
# delete files from previous runs
!rm -f hmp.parquet*

# download the file containing the data in PARQUET format
!wget https://github.com/IBM/coursera/raw/master/hmp.parquet
    
# create a dataframe out of it
df = spark.read.parquet('hmp.parquet')

# register a corresponding query table
df.createOrReplaceTempView('df')

In [2]:
from pyspark.ml.feature import VectorAssembler

vectorAssembler = VectorAssembler(inputCols=["x","y","z"],
                                  outputCol="features")

In [8]:
from pyspark.ml.clustering import KMeans

kmeans = KMeans().setK(13).setSeed(1)

In [9]:

from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[vectorAssembler, kmeans])


In [20]:
df.createOrReplaceTempView('df')
df = spark.sql("select * from df where class in ('Brush_teeth','Climb_stairs')")

In [21]:
model = pipeline.fit(df)

In [22]:
wssse = model.stages[1].computeCost(vectorAssembler.transform(df))
print("Within Set Sum of Squared Errors = " + str(wssse))

Within Set Sum of Squared Errors = 2608592.45879
